<a href="https://colab.research.google.com/github/silassjjunior/An-lise-de-Amea-as-STRIDE-com-IA-Open-Source/blob/main/Cerebro_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -----------------------------------------------------------------------------
# PASSO 1: INSTALAR TODAS AS DEPENDÊNCIAS
# -----------------------------------------------------------------------------
# Usamos o -q para uma instalação "quieta", sem poluir a saída.
!pip install -q transformers torch bitsandbytes accelerate Pillow flask pyngrok flask-cors

# -----------------------------------------------------------------------------
# PASSO 2: IMPORTAR BIBLIOTECAS E CONFIGURAR O MODELO DE IA
# -----------------------------------------------------------------------------
import torch
from transformers import AutoProcessor, LlavaForConditionalGeneration
from PIL import Image
from flask import Flask, request, jsonify
from pyngrok import ngrok, conf
from flask_cors import CORS
import os
import threading

print("✅ Dependências instaladas. Iniciando a configuração do modelo...")

# Configuração do Modelo
MODEL_ID = "llava-hf/llava-1.5-7b-hf"

# Carregando o modelo em 4-bit (quantizado) para economizar memória da GPU.
# Isso é crucial para rodar no ambiente gratuito do Colab.
model = LlavaForConditionalGeneration.from_pretrained(
    MODEL_ID,
    torch_dtype=torch.float16,
    load_in_4bit=True,
    device_map="auto"  # Mapeia o modelo para a GPU automaticamente
)

# Carregando o processador, que prepara a imagem e o texto para o modelo.
processor = AutoProcessor.from_pretrained(MODEL_ID)

print("✅ Modelo de IA e processador carregados com sucesso na GPU.")

# -----------------------------------------------------------------------------
# PASSO 3: DEFINIR A FUNÇÃO DE ANÁLISE (O CÉREBRO)
# -----------------------------------------------------------------------------
def analyze_architecture(image_bytes, system_description):
    """
    Analisa uma imagem de arquitetura usando o modelo LLaVA.
    """
    try:
        # Abre a imagem a partir dos bytes recebidos na requisição.
        raw_image = Image.open(image_bytes).convert('RGB')

        # --- ENGENHARIA DE PROMPT ---
        # Este é o coração da nossa lógica. Instruímos a IA sobre o que fazer.
        # O formato USER: <image>\n... ASSISTANT: é específico do LLaVA.
        prompt = (
            "USER: <image>\n"
            "A imagem mostra a arquitetura de um sistema. A descrição do sistema é: '{}'.\n\n"
            "Aja como um especialista em segurança de aplicações. Analise a arquitetura e a descrição "
            "para identificar possíveis ameaças de segurança com base na metodologia STRIDE (Spoofing, "
            "Tampering, Repudiation, Information Disclosure, Denial of Service, Elevation of Privilege). "
            "Retorne sua análise em um formato JSON válido. O JSON deve ter uma chave 'analysis' que contém "
            "uma lista de objetos. Cada objeto deve representar um componente da arquitetura e deve conter "
            "as chaves 'component', 'threats' (uma lista de objetos de ameaça) e 'suggestions' (uma lista de strings). "
            "Cada objeto de ameaça deve ter as chaves 'stride_category' e 'threat_description'."
            "\nASSISTANT:"
        ).format(system_description)

        # Prepara as entradas (imagem e texto) para o modelo.
        inputs = processor(text=prompt, images=raw_image, return_tensors="pt").to("cuda", torch.float16)

        # Gera a resposta da IA.
        output = model.generate(**inputs, max_new_tokens=2048)

        # Decodifica a saída e extrai apenas a parte gerada pelo assistente (o JSON).
        response_text = processor.decode(output[0], skip_special_tokens=True)
        json_part = response_text.split("ASSISTANT:")[1].strip()

        print("✅ Análise gerada pela IA.")
        return json_part

    except Exception as e:
        print(f"❌ Erro durante a análise da IA: {e}")
        return None

# -----------------------------------------------------------------------------
# PASSO 4: CRIAR A API COM FLASK E EXPÔ-LA COM NGROK
# -----------------------------------------------------------------------------
print("🚀 Iniciando o servidor Flask e o túnel Ngrok...")

# adicione seu token de autenticação do Ngrok na linha abaixo
# Você pode obtê-lo gratuitamente em https://dashboard.ngrok.com/get-started/your-authtoken
conf.get_default( ).auth_token = "COLE O SEU TOKEN AQUI" # cole o seu token

app = Flask(__name__)
CORS(app)  # Habilita CORS para permitir requisições do frontend local.

@app.route('/analyze', methods=['POST'])
def handle_analysis():
    if 'image' not in request.files:
        return jsonify({"error": "Nenhuma imagem enviada"}), 400

    image_file = request.files['image']
    description = request.form.get('description', 'Nenhuma descrição fornecida.')

    # Passamos os bytes da imagem diretamente para a função de análise.
    analysis_json_str = analyze_architecture(image_file.stream, description)

    if analysis_json_str:
        try:
            # Validamos se a string recebida é um JSON válido antes de retornar.
            analysis_data = jsonify(eval(analysis_json_str.replace("```json", "").replace("```", "")))
            print("✅ Resposta JSON enviada com sucesso.")
            return analysis_data
        except Exception as e:
            print(f"❌ Erro ao converter a resposta da IA para JSON: {e}")
            return jsonify({"error": "A IA não retornou um JSON válido.", "raw_response": analysis_json_str}), 500
    else:
        return jsonify({"error": "Falha ao gerar a análise."}), 500

# Função para rodar o app Flask em uma thread separada
def run_app():
    app.run(port=5000, host='0.0.0.0')

# Inicia o túnel Ngrok e o servidor Flask
# O Ngrok criará uma URL pública para nossa API Flask.
public_url = ngrok.connect(5000)
print(f"✅ Cérebro de IA está no ar! URL pública: {public_url}")

# Inicia o servidor Flask em uma thread para não bloquear a célula do Colab
threading.Thread(target=run_app).start()
